<!-- # Copyright (c) 2025 takotime808 -->

### Report Generation

In [ ]:
!cd .. && python multioutreg/report/generate_report1.py

In [ ]:
!cd .. && python multioutreg/report/generate_report1_safe.py

In [ ]:
# !cd .. && python examples/generate_report.py
!cd .. && python multioutreg/report/generate_report.py

In [4]:
# from jinja2 import Environment, FileSystemLoader

# env = Environment(loader=FileSystemLoader('multioutreg/report/template.html'))
# template = env.get_template('template.html')

# report_html = template.render(
#     project_title="Your Model Report",
#     model_type="Multi-fidelity Gaussian Process",
#     fidelity_levels=['Low', 'High'],
#     output_names=['Y1', 'Y2'],
#     description="Explains a multi-fidelity surrogate using X, Y...",
#     metrics={'Y1': {'r2': 0.92, 'rmse': 0.2, 'mae': 0.15}, 'Y2': {...}},
#     uncertainty_metrics={'NLL': 1.23, 'Sharpness': 0.45, ...},
#     uncertainty_plots=[{'img_b64': ..., 'title': "Calibration Curve", 'caption': "..."}],
#     prediction_plots={'Y1': ..., 'Y2': ...},
#     shap_plots={'Y1': ..., 'Y2': ...},
#     pdp_plots={'Y1': ..., 'Y2': ...},
#     sampling_umap_plot=...,
#     sampling_method_explanation="Sampling appears to be LHS, justified by...",
#     sampling_other_plots=[...],
#     other_plots=[...],
#     n_train=100,
#     n_test=40,
#     cross_validation="5-fold",
#     seed=42,
#     notes="Any extra info\nAnother note"
# )
